#**Bibliotecas**

In [142]:
# Manipulação de dados
import pandas as pd
import numpy as np
# Visualização
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
# Pré-processamento
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# Treinamento
from sklearn.model_selection import(
    train_test_split,
    GridSearchCV,
    cross_val_score,
    KFold,
    cross_validate)
# Algoritmos
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
# Avaliação
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix)
# Métricas
from sklearn.metrics import mean_squared_error, r2_score

#Ignorar alertas
import warnings
warnings.filterwarnings("ignore")

#Dados

In [143]:
#Carregar a base

df = pd.read_csv('/content/aluguel.csv')
df.head(5)

,cidade,area,quartos,banheiros,vagas,andar,animal,mobilhada,valorCondominio,valorAluguel,IPTU,seguroIncendio,totalAluguel
0,São Paulo,70,2,1,1,7,0,0,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,0,1,1200,4960,1750,63,7973
2,Porto Alegre,80,1,1,1,6,0,1,1000,2800,0,41,3841
3,Porto Alegre,51,2,1,0,2,0,1,270,1112,22,17,1421
4,São Paulo,25,1,1,0,1,1,1,0,800,25,11,836


In [144]:
# Verificar informações
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10660 entries, 0 to 10659
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cidade           10660 non-null  object
 1   area             10660 non-null  int64 
 2   quartos          10660 non-null  int64 
 3   banheiros        10660 non-null  int64 
 4   vagas            10660 non-null  int64 
 5   andar            10660 non-null  int64 
 6   animal           10660 non-null  int64 
 7   mobilhada        10660 non-null  int64 
 8   valorCondominio  10660 non-null  int64 
 9   valorAluguel     10660 non-null  int64 
 10  IPTU             10660 non-null  int64 
 11  seguroIncendio   10660 non-null  int64 
 12  totalAluguel     10660 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.1+ MB


In [145]:
# Verificar NaN
df.isna().sum()

cidade             0
area               0
quartos            0
banheiros          0
vagas              0
andar              0
animal             0
mobilhada          0
valorCondominio    0
valorAluguel       0
IPTU               0
seguroIncendio     0
totalAluguel       0
dtype: int64

In [146]:
# Descritivas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
area,10660.0,140.95,133.45,11.0,56.0,90.0,181.00,2000.0
quartos,10660.0,2.50,1.17,1.0,2.0,2.0,3.00,13.0
banheiros,10660.0,2.23,1.40,1.0,1.0,2.0,3.00,10.0
vagas,10660.0,1.60,1.58,0.0,0.0,1.0,2.00,12.0
andar,10660.0,5.01,5.28,0.0,1.0,3.0,8.00,28.0
animal,10660.0,0.22,0.42,0.0,0.0,0.0,0.00,1.0
mobilhada,10660.0,0.76,0.43,0.0,1.0,1.0,1.00,1.0
valorCondominio,10660.0,911.06,1104.39,0.0,170.0,556.5,1227.50,10000.0
valorAluguel,10660.0,3881.49,3374.12,450.0,1522.5,2650.0,5000.00,30000.0
IPTU,10660.0,325.16,539.92,0.0,38.0,125.0,375.00,5917.0


In [147]:
#Substituir os valores de vagas

df['vagasgerais'] = df['vagas'].replace({
    0: 'nao', 1: 'sim' , 2: 'sim' , 3: 'sim' , 4: 'sim' , 5: 'sim' , 6: 'sim' , 7: 'sim' , 8: 'sim' , 9: 'sim' , 10: 'sim' , 11: 'sim' , 12: 'sim'
})

df['vagasgerais']

0        sim
1        nao
2        sim
3        nao
4        nao
        ... 
10655    sim
10656    sim
10657    nao
10658    sim
10659    nao
Name: vagasgerais, Length: 10660, dtype: object

In [148]:
# Selecionar colunas numéricas
colunas_numericas = df.select_dtypes(include=['int','float']).columns
colunas_numericas

Index(['area', 'quartos', 'banheiros', 'vagas', 'andar', 'animal', 'mobilhada',
       'valorCondominio', 'valorAluguel', 'IPTU', 'seguroIncendio',
       'totalAluguel'],
      dtype='object')

In [149]:
# Selecionar colunas categóricas
colunas_categoricas = df.select_dtypes(include=['object']).columns
colunas_categoricas

Index(['cidade', 'vagasgerais'], dtype='object')

In [150]:
# Alterar para números
cols = ['area', 'quartos', 'banheiros', 'andar','totalAluguel' ]

for col in cols:
  df[col] = pd.to_numeric(df[col], errors = 'coerce')

  df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10660 entries, 0 to 10659
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cidade           10660 non-null  object
 1   area             10660 non-null  int64 
 2   quartos          10660 non-null  int64 
 3   banheiros        10660 non-null  int64 
 4   vagas            10660 non-null  int64 
 5   andar            10660 non-null  int64 
 6   animal           10660 non-null  int64 
 7   mobilhada        10660 non-null  int64 
 8   valorCondominio  10660 non-null  int64 
 9   valorAluguel     10660 non-null  int64 
 10  IPTU             10660 non-null  int64 
 11  seguroIncendio   10660 non-null  int64 
 12  totalAluguel     10660 non-null  int64 
 13  vagasgerais      10660 non-null  object
dtypes: int64(12), object(2)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10660 entries, 0 to 10659
Data columns (total 14 columns):
 #   Column

#EDA

In [151]:
# Estatísticas descritivas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
area,10660.0,140.95,133.45,11.0,56.0,90.0,181.00,2000.0
quartos,10660.0,2.50,1.17,1.0,2.0,2.0,3.00,13.0
banheiros,10660.0,2.23,1.40,1.0,1.0,2.0,3.00,10.0
vagas,10660.0,1.60,1.58,0.0,0.0,1.0,2.00,12.0
andar,10660.0,5.01,5.28,0.0,1.0,3.0,8.00,28.0
animal,10660.0,0.22,0.42,0.0,0.0,0.0,0.00,1.0
mobilhada,10660.0,0.76,0.43,0.0,1.0,1.0,1.00,1.0
valorCondominio,10660.0,911.06,1104.39,0.0,170.0,556.5,1227.50,10000.0
valorAluguel,10660.0,3881.49,3374.12,450.0,1522.5,2650.0,5000.00,30000.0
IPTU,10660.0,325.16,539.92,0.0,38.0,125.0,375.00,5917.0


In [152]:
# Médias por cidade
pd.pivot_table(df, index='cidade',
               values=colunas_numericas,
               aggfunc={'mean', 'std'}).round(2)

IPTU         andar       animal          area          \
                  mean     std  mean   std   mean   std    mean     std   
cidade                                                                    
Belo Horizonte  261.57  337.95  3.59  4.20   0.26  0.44  170.63  168.84   
Campinas        147.74  212.53  4.23  4.39   0.19  0.39  122.85  122.88   
Porto Alegre    124.09  261.64  3.91  3.87   0.16  0.36  103.66  101.55   
Rio de Janeiro  231.83  352.93  5.25  4.03   0.20  0.40  105.40   93.53   
São Paulo       429.18  653.05  5.60  5.98   0.24  0.43  153.90  136.96   

               banheiros        ... seguroIncendio        totalAluguel  \
                    mean   std  ...           mean    std         mean   
cidade                          ...                                      
Belo Horizonte      2.39  1.42  ...          53.48  53.51      4506.41   
Campinas            1.96  1.35  ...          32.41  33.21      3174.54   
Porto Alegre        1.73  1.03  ...          36.45  36.57      2964.82   
Rio de Janeiro      1.76  0.97  ...          42.47  37.54      4532.59   
São Paulo           2.46  1.50  ...          62.12  49.25      6213.93   

                        vagas       valorAluguel          valorCondominio  \
                    std  mean   std         mean      std            mean   
cidade                                                                      
Belo Horizonte  4111.07  1.95  1.66      3647.68  3479.01          543.40   
Campinas        2860.50  1.56  1.52      2365.73  2313.42          628.64   
Porto Alegre    2596.06  1.05  1.16      2339.07  2200.11          465.19   
Rio de Janeiro  3683.54  0.74  0.86      3232.21  2805.94         1026.05   
São Paulo       4797.66  1.87  1.68      4630.64  3583.52         1091.72   

                         
                    std  
cidade                   
Belo Horizonte   745.97  
Campinas         676.60  
Porto Alegre     524.00  
Rio de Janeiro   853.68  
São Paulo       1293.38  

[5 rows x 24 columns]

In [153]:
contagem_cidades = df['cidade'].value_counts()

# Exibir as cidades mais frequentes
print("As cidades que mais aparecem nos dados são:")
print(contagem_cidades.head(5))

As cidades que mais aparecem nos dados são:
cidade
São Paulo         5866
Rio de Janeiro    1499
Belo Horizonte    1252
Porto Alegre      1192
Campinas           851
Name: count, dtype: int64


In [154]:
# Classe para gerar os gráficos
class Graficos:
    def __init__(self, df):
        self.df = df

    def boxplot_por_cidade(self, col1, col_cidade='cidade'):
        fig = px.box(self.df,
                     x=col_cidade, y=col1,
                     color = col_cidade,
                     template='plotly_dark')

        fig.update_layout(xaxis_title='Cidade', yaxis_title=col1)

        fig.show()

    def barras(self, col1):
        counts = self.df[col1].value_counts()

        # Criar gráfico de barras
        fig = px.bar(x=counts.index,
                     y=counts.values,
                     text_auto = True,
                     template='plotly_dark')

        fig.update_traces(textposition='outside')

        fig.update_layout(xaxis_title=col1,
                          yaxis_title='Contagem',
                          )

        fig.show()

In [155]:
# Criar o objeto
graficos = Graficos(df)

In [156]:
# Barras
for col in colunas_categoricas:
    graficos.barras(col)

In [157]:
# Boxplot gêneros
for col in colunas_numericas:
    graficos.boxplot_por_cidade(col)

In [158]:
#Determinando as percentagens de animais aceitos com base nas variáveis.

variaveis = ['cidade']
for v in variaveis:
  resultado = df.groupby(v)['animal'].value_counts(normalize=True).unstack()
  print(f"\Determinando as percentagens de animais aceitos com base na variável'{v}' :")
  print(resultado)

\Determinando as percentagens de animais aceitos com base na variável'cidade' :
animal                 0         1
cidade                            
Belo Horizonte  0.740415  0.259585
Campinas        0.811986  0.188014
Porto Alegre    0.842282  0.157718
Rio de Janeiro  0.802535  0.197465
São Paulo       0.761848  0.238152


In [159]:

pd.pivot_table(df, index='cidade',
               values=colunas_numericas,
               aggfunc={'mean', 'std'}).round(2)

IPTU         andar       animal          area          \
                  mean     std  mean   std   mean   std    mean     std   
cidade                                                                    
Belo Horizonte  261.57  337.95  3.59  4.20   0.26  0.44  170.63  168.84   
Campinas        147.74  212.53  4.23  4.39   0.19  0.39  122.85  122.88   
Porto Alegre    124.09  261.64  3.91  3.87   0.16  0.36  103.66  101.55   
Rio de Janeiro  231.83  352.93  5.25  4.03   0.20  0.40  105.40   93.53   
São Paulo       429.18  653.05  5.60  5.98   0.24  0.43  153.90  136.96   

               banheiros        ... seguroIncendio        totalAluguel  \
                    mean   std  ...           mean    std         mean   
cidade                          ...                                      
Belo Horizonte      2.39  1.42  ...          53.48  53.51      4506.41   
Campinas            1.96  1.35  ...          32.41  33.21      3174.54   
Porto Alegre        1.73  1.03  ...          36.45  36.57      2964.82   
Rio de Janeiro      1.76  0.97  ...          42.47  37.54      4532.59   
São Paulo           2.46  1.50  ...          62.12  49.25      6213.93   

                        vagas       valorAluguel          valorCondominio  \
                    std  mean   std         mean      std            mean   
cidade                                                                      
Belo Horizonte  4111.07  1.95  1.66      3647.68  3479.01          543.40   
Campinas        2860.50  1.56  1.52      2365.73  2313.42          628.64   
Porto Alegre    2596.06  1.05  1.16      2339.07  2200.11          465.19   
Rio de Janeiro  3683.54  0.74  0.86      3232.21  2805.94         1026.05   
São Paulo       4797.66  1.87  1.68      4630.64  3583.52         1091.72   

                         
                    std  
cidade                   
Belo Horizonte   745.97  
Campinas         676.60  
Porto Alegre     524.00  
Rio de Janeiro   853.68  
São Paulo       1293.38  

[5 rows x 24 columns]

In [160]:
#Cálculo da média do número de banheiros em imóveis com animais em relação aos
#que não tem

pd.pivot_table(df, index='animal',
               values='banheiros',
               aggfunc={'mean', 'std'}).round(2)

,mean,std
animal,,
0,2.32,1.42
1,1.92,1.28


In [161]:
# Calculando a média da área dos imóveis
media_por_cidade = df.groupby('cidade')['area'].mean().reset_index()

# Criando o gráfico de barras
fig = px.bar(media_por_cidade, x='cidade', y='area',
             template='plotly_dark',
             title='Média da área dos imóveis para alugar em cada cidade')

# Exibindo o gráfico
fig.show()

In [162]:
#Correlaçao de todos os dados
numeric_df = df.select_dtypes(include='number')

# Calcula a matriz de correlação entre todas as variáveis numéricas
corr = numeric_df.corr()

corr_styled = corr.style.background_gradient()


corr_styled

,area,quartos,banheiros,vagas,andar,animal,mobilhada,valorCondominio,valorAluguel,IPTU,seguroIncendio,totalAluguel
area,1.000000,0.693846,0.754329,0.689691,-0.110507,-0.130282,0.007595,0.313331,0.665227,0.650499,0.700988,0.673153
quartos,0.693846,1.000000,0.734056,0.616524,-0.092299,-0.172587,0.080455,0.287895,0.543797,0.485068,0.567905,0.552452
banheiros,0.754329,0.734056,1.000000,0.697518,0.005831,-0.120138,-0.018603,0.418714,0.669713,0.609185,0.677914,0.697626
vagas,0.689691,0.616524,0.697518,1.000000,-0.028029,-0.128840,0.004562,0.295595,0.578289,0.559632,0.597872,0.590146
andar,-0.110507,-0.092299,0.005831,-0.028029,1.000000,0.027939,-0.124369,0.354266,0.083224,0.075677,0.014537,0.161457
animal,-0.130282,-0.172587,-0.120138,-0.128840,0.027939,1.000000,-0.088928,-0.022600,-0.067654,-0.081133,-0.079325,-0.068073
mobilhada,0.007595,0.080455,-0.018603,0.004562,-0.124369,-0.088928,1.000000,-0.127906,-0.165710,-0.039644,-0.143387,-0.164846
valorCondominio,0.313331,0.287895,0.418714,0.295595,0.354266,-0.022600,-0.127906,1.000000,0.455289,0.542878,0.370370,0.667768
valorAluguel,0.665227,0.543797,0.669713,0.578289,0.083224,-0.067654,-0.165710,0.455289,1.000000,0.606223,0.987144,0.961875
IPTU,0.650499,0.485068,0.609185,0.559632,0.075677,-0.081133,-0.039644,0.542878,0.606223,1.000000,0.592981,0.727008


In [163]:
# Calcular a média do preço do aluguel para cada grupo
media_aluguel = df.groupby('animal')['totalAluguel'].mean().reset_index()

# Criar um gráfico de barras com Plotly Express
fig = px.bar(media_aluguel, x='animal', y='totalAluguel',
             labels={'animal': 'Permite Animais', 'totalAluguel': 'Média do Preço do Aluguel'},
             template='plotly_dark',
             title='Média do Preço do Aluguel por Permissão de Animais de Estimação',
           )

fig.show()

In [164]:
# Calculando a média de preço de aluguel em relação com a cidade
media_por_cidade = df.groupby('cidade')['totalAluguel'].mean().reset_index()

# Criando o gráfico de barras
fig = px.bar(media_por_cidade, x='cidade', y='totalAluguel',
             template='plotly_dark',
             labels={'cidade': 'Cidades', 'totalAluguel': 'Média de preço dos aluguéis'},
             title='Média de preço de aluguel em relação com a cidade')

# Exibindo o gráfico
fig.show()

In [165]:
# Calculando a média de preço de aluguel em relação com a cidade
media_por_cidade = df.groupby('vagas')['totalAluguel'].mean().reset_index()

# Criando o gráfico de barras
fig = px.bar(media_por_cidade, x='vagas', y='totalAluguel',
             template='plotly_dark',
             labels={'vagas': 'vagas', 'totalAluguel': 'valor do aluguel'},
             title='Média dos preços de alugueis em relação a quantidade de vagas.')

# Definindo todos os números no eixo x
fig.update_xaxes(tickmode='array', tickvals=list(media_por_cidade['vagas']))

# Exibindo o gráfico
fig.show()

In [166]:
# Relacionar a proporção com o preço do aluguel em cada cidade
media_por_cidade_com_vaga = df[df['vagas'] > 0].groupby('cidade').agg({'totalAluguel': 'mean'})
media_por_cidade_sem_vaga = df[df['vagas'] == 0].groupby('cidade').agg({'totalAluguel': 'mean'})

print("\nMédia do preço do aluguel em cada cidade para imóveis com vaga de garagem:")
print(media_por_cidade_com_vaga)
print("\nMédia do preço do aluguel em cada cidade para imóveis sem vaga de garagem:")
print(media_por_cidade_sem_vaga)


Média do preço do aluguel em cada cidade para imóveis com vaga de garagem:
                totalAluguel
cidade                      
Belo Horizonte   4783.889727
Campinas         3454.652738
Porto Alegre     3615.305960
Rio de Janeiro   5802.353228
São Paulo        7061.853213

Média do preço do aluguel em cada cidade para imóveis sem vaga de garagem:
                totalAluguel
cidade                      
Belo Horizonte   2965.020942
Campinas         1936.337580
Porto Alegre     1840.972540
Rio de Janeiro   2995.010324
São Paulo        2961.327288


In [167]:
# Relacionar a proporção com o preço do aluguel em cada cidade
media_por_cidade_com_vaga = df[df['vagas'] > 0].groupby('cidade').agg({'totalAluguel': 'mean'})
media_por_cidade_sem_vaga = df[df['vagas'] == 0].groupby('cidade').agg({'totalAluguel': 'mean'})

# Criar gráfico para imóveis com vaga de garagem
fig1 = px.bar(media_por_cidade_com_vaga, x=media_por_cidade_com_vaga.index, y='totalAluguel',
               title='Média do Preço do Aluguel em Cada Cidade para Imóveis com Vaga de Garagem',
               labels={'totalAluguel': 'Preço do Aluguel', 'cidade': 'Cidade'}, template='plotly_dark')

# Criar gráfico para imóveis sem vaga de garagem
fig2 = px.bar(media_por_cidade_sem_vaga, x=media_por_cidade_sem_vaga.index, y='totalAluguel',
               title='Média do Preço do Aluguel em Cada Cidade para Imóveis sem Vaga de Garagem',
               labels={'totalAluguel': 'Preço do Aluguel', 'cidade': 'Cidade'}, template='plotly_dark')

# Exibir os gráficos
fig1.show()
fig2.show()

# Treinamento e Processamento


In [168]:
# Substitui no DataFrame
df['cidade'] = df['cidade'].replace({
    'São Paulo': 1,
    'Campinas': 2,
    'Porto Alegre': 3,
    'Belo Horizonte': 4,
    'Rio de Janeiro': 5
})

df.drop('vagasgerais',axis=1, inplace=True)

df.head(5)

,cidade,area,quartos,banheiros,vagas,andar,animal,mobilhada,valorCondominio,valorAluguel,IPTU,seguroIncendio,totalAluguel
0,1,70,2,1,1,7,0,0,2065,3300,211,42,5618
1,1,320,4,4,0,20,0,1,1200,4960,1750,63,7973
2,3,80,1,1,1,6,0,1,1000,2800,0,41,3841
3,3,51,2,1,0,2,0,1,270,1112,22,17,1421
4,1,25,1,1,0,1,1,1,0,800,25,11,836


In [179]:
# Criar X e y
X = df.iloc[:,: -1]
y = df.iloc[:,-1]


In [170]:
# Codificar alvo
le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
# Armazenar os valores reais de y
labels = list(le.classes_)
labels


In [172]:
# Criar o pipeline para escalonar os dados
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('modelo', None)
])

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state = 42)

In [173]:
# Definir uma grade de hiperparâmetros
param_grid = [{
    'modelo': [RandomForestRegressor(random_state =42)],
    'modelo__n_estimators': [50, 100, 150],
    'modelo__max_depth': [5, 10, 15],

    },
    {
    'modelo': [DecisionTreeRegressor(random_state=42)],
    'modelo__max_depth': [3,5,7,10],
    'modelo__min_samples_split': [2,5,10]
    },
    {
    'modelo': [LinearRegression()],
    'modelo__fit_intercept': ['True', 'False'],
    }
]

In [174]:
# Criar o objeto Grid Search
grid_search = GridSearchCV(pipeline, param_grid, cv = 3,
                           scoring = 'accuracy',
                           verbose =1,
                           n_jobs = -1)


In [175]:
# Treinar
grid_search.fit(X_train, y_train)

print(f"Melhores hiperparâmetros:"
      f"{grid_search.best_params_}")

Fitting 3 folds for each of 23 candidates, totalling 69 fits
Melhores hiperparâmetros:{'modelo': RandomForestRegressor(max_depth=5, n_estimators=50, random_state=42), 'modelo__max_depth': 5, 'modelo__n_estimators': 50}


In [176]:
# Melhor modelo (hiperparâmetros)
melhor_modelo = grid_search.best_estimator_
melhor_modelo

Pipeline(steps=[('scaler', StandardScaler()),
                ('modelo',
                 RandomForestRegressor(max_depth=5, n_estimators=50,
                                       random_state=42))])

In [184]:
# testar
rf = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=42)
rf.fit(X_train, y_train)
y_pred = melhor_modelo.predict(X_test)

In [178]:
# Avaliar o modelo
mse = mean_squared_error(y_test,y_pred)
r2  = r2_score(y_test,y_pred)
print(f"Erro Quadrático Médio (Teste): {mse.round(2)}\n"
      f"R2 (Teste): {r2.round(2)}")

Erro Quadrático Médio (Teste): 26660.17
R2 (Teste): 0.99


In [180]:
# Criar figura
fig = go.Figure()

In [181]:
# Adicionar linha de tendência (valores reais)
fig.add_trace(go.Scatter(x=y_test,
                         y=y_test,
                         mode='lines',
                         name='Real',
                         line=dict(color='blue')))

# Adicionar pontos para valores previstos
fig.add_trace(go.Scatter(x=y_test,
                         y=y_pred,
                         mode='markers',
                         name='Previsto',
                         marker=dict(color='red')))

# Layout do gráfico
fig.update_layout(title='Comparação entre Valores Reais e Previstos',
                  xaxis_title='Valores Reais',
                  yaxis_title='Valores Previstos',
                  showlegend=True)

# Exibir gráfico
fig.show()

In [186]:
# Obter a importância das características
importancias = rf.feature_importances_

In [187]:
# Montar a equação com as características mais importantes
equacao_rf = 'totalAluguel = '
for i in range(len(df.columns[:-1])):
    equacao_rf += f'{importancias[i]:.2f} * {df.columns[i]} + '

In [188]:
# Montar a equação com as características mais importantes
equacao_rf = 'totalAluguel = '
for i in range(len(df.columns[:-1])):
    equacao_rf += f'{importancias[i]:.2f} * {df.columns[i]} + '


In [189]:
# Remover o último sinal de adição
equacao_rf = equacao_rf

equacao_rf[:-3]


'totalAluguel = 0.00 * cidade + 0.00 * area + 0.00 * quartos + 0.00 * banheiros + 0.00 * vagas + 0.00 * andar + 0.00 * animal + 0.00 * mobilhada + 0.06 * valorCondominio + 0.94 * valorAluguel + 0.00 * IPTU + 0.00 * seguroIncendio'

In [190]:
# Nome das características
nomes_caracteristicas = df.columns[:-1]

# Criar o gráfico de radar
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=importancias,
      theta=nomes_caracteristicas,
      fill='toself',
      name='Importância das Características'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, max(importancias)]
    )),
  showlegend=True
)

fig.show()